In [ ]:
pip install requests orjson

This benchmark compares the performance of the json.loads() and orjson.loads() functions when deserializing JSON data obtained from the SEC. The results show that orjson is significantly faster than the standard json library, making it a preferable choice for our application requiring efficient processing of large volumes of JSON data.

In [ ]:
import time
import requests
import json
import orjson

url = "https://data.sec.gov/api/xbrl/companyfacts/CIK0001318605.json"
headers = {
    "User-Agent": "FinDrum Contact <[email protected]>"
}

response = requests.get(url, headers=headers)
response.raise_for_status()
content_bytes = response.content
content_str = content_bytes.decode('utf-8')

start = time.time()
data_json = json.loads(content_str)
elapsed_json = time.time() - start
print(f"json.loads: {elapsed_json:.4f} seconds")

start = time.time()
data_orjson = orjson.loads(content_bytes)
elapsed_orjson = time.time() - start
print(f"orjson.loads: {elapsed_orjson:.4f} seconds")

print(f"Are the results equal? {data_json == data_orjson}")